In [3]:
import pandas as pd
pd.set_option('display.max_rows', 130, 'display.max_columns', 130)
pd.options.display.float_format = '{:,.2f}'.format

import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
# file name or path to be used for code 
loanfile = Desktop/Lending Case Study/ loan.csv
data = pd.read_csv(loanfile, error_bad_lines=False, index_col=False, dtype='unicode')
data.head()
data.shape
data.info()
percent_missing = data.isnull().sum() * 100 / len(data)
missing_value_df = pd.DataFrame({'column_name': data.columns,
                                 'percent_missing': percent_missing.round(2)})
missing_value_df.sort_values('percent_missing', inplace=True)
# It was found that ~50 columns had null values for all variables
(missing_value_df['percent_missing'] == 100.00).sum()
# the list of columns having NULL values
missing_values_columns = missing_value_df[(missing_value_df['percent_missing'] == 100.00)].column_name.tolist()
# lets remove the columns having missing values
null_columns_dropped_data = data.drop(missing_values_columns, axis=1)
null_columns_dropped_data = null_columns_dropped_data.drop(['desc', 'member_id', 'id', 'url', 'next_pymnt_d',
                                                           'mths_since_last_record', 'mths_since_last_delinq'], axis=1)
## Delete columns having all values same. Not useful in analysis
null_columns_dropped_data = null_columns_dropped_data.drop(['delinq_amnt', 'acc_now_delinq', 'application_type',
                                                           'policy_code', 'pymnt_plan', 'initial_list_status', 'chargeoff_within_12_mths',
                                                           'tax_liens', 'collections_12_mths_ex_med'], axis=1)
## Deleting string columns having most values as unique values
null_columns_dropped_data = null_columns_dropped_data.drop(['emp_title'], axis=1)
## Check percentage of missing data
((null_columns_dropped_data.isnull().sum() * 100 / len(null_columns_dropped_data)).sort_values(
    ascending=False)).head(11)
null_columns_dropped_data.shape

# find the categorical columns
data = []
for row in null_columns_dropped_data.columns.tolist():
    data.append({'column': row, 'count': len(null_columns_dropped_data[row].unique())})
unique = pd.DataFrame(data).sort_values('count')
unique.head(19)
#Create derived columns
# null_columns_dropped_data[null_columns_dropped_data.isnull().sum(axis=1) >= 2]
null_columns_dropped_data['term_months'] = null_columns_dropped_data['term'].str.rstrip('months')
del null_columns_dropped_data['term']
null_columns_dropped_data['int_rate_percentage'] = null_columns_dropped_data['int_rate'].str.rstrip('%')
del null_columns_dropped_data['int_rate']
null_columns_dropped_data['emp_length_years'] = null_columns_dropped_data['emp_length'].str.rstrip('years ')
null_columns_dropped_data['emp_length_years'].replace({'10+': '10'}, inplace=True)
null_columns_dropped_data['emp_length_years'][null_columns_dropped_data['emp_length_years'] == '< 1'] = '0.5'
del null_columns_dropped_data['emp_length']
null_columns_dropped_data['revol_util_percentage'] = null_columns_dropped_data['revol_util'].str.rstrip('%')
del null_columns_dropped_data['revol_util']
null_columns_dropped_data['issue_d_month'], null_columns_dropped_data['issue_d_year'] = null_columns_dropped_data[
    'issue_d'].str.split('-').str

null_columns_dropped_data['last_credit_pull_d_month'], null_columns_dropped_data['last_credit_pull_d_year'] = null_columns_dropped_data[
    'last_credit_pull_d'].str.split('-').str

null_columns_dropped_data['last_pymnt_d_month'], null_columns_dropped_data['last_pymnt_d_year'] = null_columns_dropped_data[
    'last_pymnt_d'].str.split('-').str

null_columns_dropped_data['earliest_cr_line_month'], null_columns_dropped_data['earliest_cr_line_year'] = null_columns_dropped_data[
    'earliest_cr_line'].str.split('-').str
null_columns_dropped_data['issue_d_year'] = '20' + null_columns_dropped_data['issue_d_year']
# Update of column datatype
columns = ['loan_amnt', 'funded_amnt','funded_amnt_inv', 'installment', 'annual_inc', 'emp_length_years', 'dti', 
          'revol_bal', 'out_prncp', 'out_prncp_inv', 'total_pymnt', 'total_pymnt_inv', 'total_rec_prncp', 'total_rec_int',
          'total_rec_late_fee', 'recoveries', 'collection_recovery_fee', 'int_rate_percentage', 'emp_length_years',
          'revol_util_percentage']
null_columns_dropped_data[columns] = null_columns_dropped_data[columns].astype(np.float)
# Create a profit and loss column
null_columns_dropped_data['PnL']=(
    null_columns_dropped_data['total_pymnt']-null_columns_dropped_data['funded_amnt']
)*100/null_columns_dropped_data['funded_amnt']
null_columns_dropped_data['loan_inc_ratio'
                         ] = null_columns_dropped_data.funded_amnt*100/null_columns_dropped_data.annual_inc
columns = ['total_acc', 'term_months', 'issue_d_year']
null_columns_dropped_data[columns] = null_columns_dropped_data[columns].astype(np.int)
columns = ['grade', 'sub_grade', 'home_ownership', 'verification_status', 'loan_status', 'purpose', 'addr_state', 
           'delinq_2yrs', 'inq_last_6mths', 'open_acc', 'pub_rec', 'pub_rec_bankruptcies']
null_columns_dropped_data[columns] = null_columns_dropped_data[columns].apply(lambda x: x.astype('category'))
null_columns_dropped_data['annual_inc_lakhs'] = null_columns_dropped_data['annual_inc'] / 1000
null_columns_dropped_data['annual_inc_lakhs'].describe()
null_columns_dropped_data.head()
#Count Categorical and Numeric columns
numeric_data = null_columns_dropped_data.select_dtypes(include = [np.number])

categorical_data = null_columns_dropped_data.select_dtypes(exclude = [np.number, np.object])

string_data = null_columns_dropped_data.select_dtypes(include = [np.object])

print('There are {0} numerical , {1} categorical and {2} string features in the training data'.\
     format(numeric_data.shape[1], categorical_data.shape[1], string_data.shape[1]))
#Remove current loan status as it's not needed in the analysis
null_columns_dropped_data = null_columns_dropped_data[null_columns_dropped_data.loan_status !='Current']
null_columns_dropped_data = null_columns_dropped_data.loc[:,null_columns_dropped_data.nunique()!=1]
## Final dataset for Exploratory Data Analytics
df = null_columns_dropped_data
df["loan_status"].value_counts()
# Performing Univariate Analysis
df["loan_status"].value_counts()
sns.boxplot( y=df["loan_amnt"] )
plt.ylabel('Loan Amount')
plt.show()

sns.countplot(x= null_columns_dropped_data["loan_status"])
plt.show()
df["loan_status"].value_counts()
sns.countplot(x='grade', data=df)
# Most of the loans lies in grade of A and B. Majority of loans are of high grade.
plt.show()

sns.countplot(x='emp_length_years', data=df)
plt.xlabel('Employee Working Experience')
plt.show()
sns.countplot(x='emp_length_years', hue='loan_status', data=df)
plt.xlabel('Employee Working Experience')
plt.show()
sns.distplot(df['funded_amnt'])
# Mostly the loan amount dispensed is close to 5 lakhs
plt.show()
sns.boxplot(x=df['annual_inc_lakhs'])
plt.xlabel('Annual Income in Lakhs')
# Two applicants have income > 30 Lakhs
plt.show()
sns.distplot(df[df['annual_inc_lakhs'] < 3000]['annual_inc_lakhs'], bins=5)
plt.xlabel('Annual Income in Lakhs')
#Majority of applicatns have income < 5 lakhs
plt.show()
# Relationship of Funded Amount and defaults
plt.subplot(1,2,1)
plt.title('Default')
sns.boxplot(y=df[df.loan_status=='Charged Off'].PnL)


plt.subplot(1,2,2)
plt.title('Fully Paid')
sns.boxplot(y=df[df.loan_status=='Fully Paid'].PnL)
plt.show()
sns.boxplot(y='int_rate_percentage', x='grade', data=df)
plt.ylabel('Interest Rate Percentage')
plt.show()
sns.boxplot(y='int_rate_percentage', x='loan_status', data=df)
plt.ylabel('Interest Rate Percentage')
plt.xlabel('Loan Status')
plt.show()
# Segmented Univariate Analysis
sns.countplot(x='purpose', hue="loan_status", data=df)
plt.xticks(rotation=60)
# Major of loan assigned for the debt consolidation has been eventually fully paid
plt.show()
sns.boxplot( y="loan_amnt", x='loan_status' , data=df)
plt.ylabel('Loan Amount')
plt.show()
sns.countplot( x="term_months", hue='loan_status', data=df)
plt.xlabel('Loan Term (Months)')
plt.show()
plt.figure(figsize=(10,7))
sns.countplot( x="term_months", hue='purpose', data=df)
plt.xlabel('Loan Term in Months')
# 36 months loan period - highest probability of default
plt.show()
sns.countplot(x='purpose', hue='loan_status', data=df)
plt.title('No of loans granted for various purpose')
plt.xticks(rotation=90)
plt.show()
sns.countplot(x='home_ownership',hue='loan_status', data=df)
plt.xticks(rotation=90)
plt.show()
# Bivariate Analysis
sns.barplot(y='annual_inc_lakhs', x='pub_rec_bankruptcies', data=df)
plt.xticks(rotation=60)
plt.show()
plt.figure(figsize=(10,10))
sns.heatmap(df.corr())
plt.show()
del numeric_data['out_prncp']
del numeric_data['out_prncp_inv']
plt.figure(figsize=(15,40))


for i in range(len(numeric_data.columns)):
    plt.subplot(8,3, i + 1)
    sns.boxplot(y=numeric_data.columns[i], x='loan_status', data=df)

plt.show()
g = df[df['loan_status']=='Charged Off'].groupby('addr_state')['loan_status'].count().reset_index()
plt.figure(figsize=(10,10))
sns.barplot(y='addr_state', x='loan_status', data=g)
plt.xlabel('Count of loan status to be defaulter')
plt.ylabel('State')
# Applicants from Calafornia have highest probability of defaults
plt.show()
g = df.groupby('issue_d_year')['loan_status'].count()

g.plot.line(x_compat=True)
plt.xticks(np.arange(min(g.index), max(g.index)+1, 1.0))
plt.title('No of loan granted over the years')
plt.xlabel('Loan Issue Year')
# There has been rise in loan applicantions in between 2007 to 2011
plt.show()
# Relationship of default and funded amount - barplot
sns.barplot(x='loan_status',y='PnL',data=df)
plt.xlabel("Loan Status")
plt.ylabel("Profit and Loss")

plt.title("Profit n Loss vs status relationship")
plt.show()
plt.figure(figsize=(10,10))

sns.barplot(x='loan_status',y='loan_inc_ratio',hue='purpose',data=df)
plt.xlabel('Loan Status')
plt.ylabel('Ratio of loan granted vs annual salary')
plt.show()
df_agg = df[df['funded_amnt'] <= 7000].groupby(['loan_status'])['funded_amnt'].count().reset_index()
sns.barplot(x='loan_status', y='funded_amnt', data=df_agg)
plt.title('Loan Amount granted less than 7 lakhs')
plt.xlabel('Loan Status')
plt.ylabel('Loan Amount Granted')
plt.show()
df_agg = df[df['funded_amnt'] > 7000].groupby(['loan_status'])['funded_amnt'].count().reset_index()
sns.barplot(x='loan_status', y='funded_amnt', data=df_agg)
plt.title('Loan Amount granted more than 7 lakhs')
plt.xlabel('Loan Status')
plt.ylabel('Loan Amount Granted')
plt.show()

FileNotFoundError: [Errno 2] No such file or directory: 'Desktop/Upgrad IIT B PG programme/Lending Case Study/loan.csv'